In [2]:
conda install scikit-learn

Solving environment: done

## Package Plan ##

  environment location: /anaconda3/envs/newenv

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    intel-openmp-2019.4        |              233         1.1 MB
    llvm-openmp-10.0.0         |       h28b9765_0         270 KB
    mkl-2019.4                 |              233       155.2 MB
    mkl-service-2.3.0          |   py38h9ed2024_0          44 KB
    mkl_fft-1.3.0              |   py38ha059aab_0         181 KB
    mkl_random-1.1.1           |   py38h959d312_0         337 KB
    numpy-1.19.2               |   py38h456fd55_0          20 KB
    numpy-base-1.19.2          |   py38hcfb5961_0         5.1 MB
    scikit-learn-0.24.1        |   py38hb2f4e1b_0         6.6 MB
    scipy-1.6.1                |   py38h2515648_0        18.6 MB
    threadpoolctl-2.1.0        |     pyh5ca1d4c_0          16 KB


In [4]:
pip install seaborn

     |████████████████████████████████| 285 kB 110 kB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 583 kB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 519 kB/s eta 0:00:01
     |████████████████████████████████| 61 kB 282 kB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports required packages

import os
import glob #to read batch of images at once
import datetime

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Create a dictionary with configuration/setup we will be using

config={
  "model"           : "vgg16",
  "weights"         : "imagenet", 
  "include_top"     : False, #Don't include the last dense layer

  "train_path"      : "/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/training",
  "test_path"       : "/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/validation",
  "features_path"   : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/features.h5",
  "labels_path"     : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/labels.h5",
  "results"         : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/results.txt",
  "classifier_path" : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/classifier.pickle",
  "model_path"      : "/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/model",

  "test_size"       : 0.20,
  "seed"            : 9,
  "num_classes"     : 2
}

In [6]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning) #Suppress Pandas Future warning 

In [7]:
# config variables
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]

In [8]:
# Here we are loading the base VGG16 model with weights and then excluding the top dense layer
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(base_model.input,base_model.get_layer('fc1').output)
    image_size = (224, 224)
else:
    base_model = None

print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


In [9]:
#for layer in base_model.layers:
    #print(layer.name)
#layer_output = model.get_layer('vgg16').get_layer('block3_conv1').output

In [12]:
#path to training dataset
#train_labels = os.listdir(train_path)
#print(train_labels)

['.DS_Store', '01-whole', '00-damage']


In [36]:
'''
img = image.load_img('/Users/rishitdholakia/Desktop/Car Damage Detection project/data1a/training/01-whole/0001.jpg', target_size=image_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
feature = model.predict(x)
print(feature)
flat = feature.flatten()
print(flat)
'''

[[10.273586  0.        0.       ...  0.        0.        2.312586]]
[10.273586  0.        0.       ...  0.        0.        2.312586]


In [24]:
# path to training dataset
train_labels = os.listdir(train_path)

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x) #2D
        flat = feature.flatten() # to 1D
        features.append(flat)
        labels.append(label)
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("Training labels: {}".format(le_labels))
print ("Training labels shape: {}".format(le_labels.shape))

Completed label - .DS_Store
Completed label - 01-whole
Completed label - 00-damage
Training labels: [1 1 1 ... 0 0 0]
Training labels shape: (1840,)


In [38]:
le_labels

array([1, 1, 1, ..., 0, 0, 0])

In [37]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [42]:
#https://www.christopherlovell.co.uk/blog/2016/04/27/h5py-intro.html
#https://machinelearningmastery.com/save-load-keras-deep-learning-models/ -- saving model architecture using JSON and weights using .h5
#https://stackoverflow.com/questions/58092765/how-to-save-list-of-extracted-features-in-hdf5-file-python

# save features and labels
h5f_data = h5py.File(features_path, 'w') # truncate if the file exists
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("Saved model and weights to disk..")

print ("Features and labels saved..")

# end time
import time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

Saved model and weights to disk..
Features and labels saved..
End time - 2021-03-20 12:19
